In [1]:
import pandas as pd
import glob
import re

In [52]:
ruta_archivo = r'C:\Users\julio\Desktop\The_Bridge\Temario\EDA\Dataset\Banco_de_Espanya\Tipo_de_cambio\be0116.csv'

# Cargar el archivo en un DataFrame saltando las primeras 5 filas
df = pd.read_csv(ruta_archivo, sep=',', encoding='latin1', skiprows=4)
df = df.iloc[1:-2]

df['year'] = df['DESCRIPCIÓN DE LAS UNIDADES'].str[-4:].astype(int)
df = df[(df['year'] >= 1995) & (df['year'] <= 2022)]

df = df.iloc[:, [0, 1, 2, 11]]

df

,DESCRIPCIÓN DE LAS UNIDADES,Dólares de Estados Unidos por Euro,Yenes japoneses por Euro,year
193,ENE 1995,1.2408,123.7433,1995
194,FEB 1995,1.2587,123.6368,1995
195,MAR 1995,1.3171,119.2986,1995
196,ABR 1995,1.3414,112.4997,1995
197,MAY 1995,1.3216,112.2487,1995
...,...,...,...,...
524,AGO 2022,1.0128,136.8548,2022
525,SEP 2022,0.9904,141.5682,2022
526,OCT 2022,0.9826,144.7252,2022
527,NOV 2022,1.0201,145.1241,2022


In [94]:
ruta_archivo = r'C:\Users\julio\Desktop\The_Bridge\Temario\EDA\Dataset\Banco_de_Espanya\Tipo_de_cambio\be2003.csv'

# Cargar el archivo en un DataFrame saltando las primeras 5 filas
df2 = pd.read_csv(ruta_archivo, sep=',', encoding='latin1', skiprows=4)
df2 = df2.iloc[1:-2]

df2 = df2.iloc[:, [0, 4]]
df2 = df2[df2.iloc[:, 1] != '_']
df2['year'] = df2['DESCRIPCIÓN DE LAS UNIDADES'].str[-4:].astype(int)


merged_df = pd.merge(df, df2, on='DESCRIPCIÓN DE LAS UNIDADES', how='left')
new_order = ['DESCRIPCIÓN DE LAS UNIDADES', 'Dólares de Estados Unidos por Euro', 'Yenes japoneses por Euro', 'Yuans por Euro']  # Define el nuevo orden de las columnas
df_tc = merged_df.reindex(columns=new_order)

df_tc

,DESCRIPCIÓN DE LAS UNIDADES,Dólares de Estados Unidos por Euro,Yenes japoneses por Euro,Yuans por Euro
0,ENE 1995,1.2408,123.7433,NaN
1,FEB 1995,1.2587,123.6368,NaN
2,MAR 1995,1.3171,119.2986,NaN
3,ABR 1995,1.3414,112.4997,NaN
4,MAY 1995,1.3216,112.2487,NaN
...,...,...,...,...
331,AGO 2022,1.0128,136.8548,6.8884
332,SEP 2022,0.9904,141.5682,6.9508
333,OCT 2022,0.9826,144.7252,7.0687
334,NOV 2022,1.0201,145.1241,7.3171


In [95]:
# Diccionario para mapear las abreviaturas de los meses a sus equivalentes en inglés
meses = {
    'ENE': 'Jan',
    'FEB': 'Feb',
    'MAR': 'Mar',
    'ABR': 'Apr',
    'MAY': 'May',
    'JUN': 'Jun',
    'JUL': 'Jul',
    'AGO': 'Aug',
    'SEP': 'Sep',
    'OCT': 'Oct',
    'NOV': 'Nov',
    'DIC': 'Dec'
}

# Reemplazar las abreviaturas de los meses con sus equivalentes en inglés
for mes_abrev, mes_ingles in meses.items():
    df_tc['DESCRIPCIÓN DE LAS UNIDADES'] = df_tc['DESCRIPCIÓN DE LAS UNIDADES'].str.replace(mes_abrev, mes_ingles)

# Convertir a formato de fecha
df_tc['DESCRIPCIÓN DE LAS UNIDADES'] = pd.to_datetime(df_tc['DESCRIPCIÓN DE LAS UNIDADES'], format='%b %Y')

df_tc.iloc[:, 1:4] = df_tc.iloc[:, 1:4].apply(pd.to_numeric, errors='coerce')

df_tc['Yuans por Euro'].fillna(0, inplace=True)




In [96]:
df_tc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 336 entries, 0 to 335
Data columns (total 4 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   DESCRIPCIÓN DE LAS UNIDADES         336 non-null    datetime64[ns]
 1   Dólares de Estados Unidos por Euro  336 non-null    float64       
 2   Yenes japoneses por Euro            336 non-null    float64       
 3   Yuans por Euro                      336 non-null    float64       
dtypes: datetime64[ns](1), float64(3)
memory usage: 10.6 KB


In [101]:
# Convertir la columna 'DESCRIPCIÓN DE LAS UNIDADES' a tipo datetime si no lo está
df_tc['DESCRIPCIÓN DE LAS UNIDADES'] = pd.to_datetime(df_tc['DESCRIPCIÓN DE LAS UNIDADES'])

# Extraer el año de la columna 'DESCRIPCIÓN DE LAS UNIDADES'
df_tc['Año'] = df_tc['DESCRIPCIÓN DE LAS UNIDADES'].dt.year

# Calcular el tipo de cambio promedio por año para cada moneda
tipos_cambio_anual = df_tc.groupby('Año').agg({
    'Dólares de Estados Unidos por Euro': 'mean',
    'Yenes japoneses por Euro': 'mean',
    'Yuans por Euro': 'mean'
})

# Mostrar los tipos de cambio promedio por año
tipos_cambio_anual


,Dólares de Estados Unidos por Euro,Yenes japoneses por Euro,Yuans por Euro
Año,,,
1995,1.308275,122.904200,0.000000
1996,1.269475,138.036508,0.000000
1997,1.134292,137.162292,0.000000
1998,1.120450,146.364050,0.000000
1999,1.066842,121.433358,0.000000
2000,0.924033,99.530075,6.948933
2001,0.895650,108.733242,7.413183
2002,0.944908,118.064625,7.820983
2003,1.130900,130.964083,9.360275


In [102]:
tipos_cambio_anual.info()

<class 'pandas.core.frame.DataFrame'>
Index: 28 entries, 1995 to 2022
Data columns (total 3 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Dólares de Estados Unidos por Euro  28 non-null     float64
 1   Yenes japoneses por Euro            28 non-null     float64
 2   Yuans por Euro                      28 non-null     float64
dtypes: float64(3)
memory usage: 784.0 bytes
